In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re

In [2]:
path = os.getcwd()
# find files
files = sorted(os.listdir(os.path.join(path,"excel")))
print(*files)

TOP-2000-1999.xlsx TOP-2000-2000.xlsx TOP-2000-2001.xlsx TOP-2000-2002.xlsx TOP-2000-2003.xlsx TOP-2000-2004.xlsx TOP-2000-2005.xlsx TOP-2000-2006.xlsx TOP-2000-2007.xlsx TOP-2000-2008.xlsx TOP-2000-2009.xlsx TOP-2000-2010.xlsx TOP-2000-2011.xlsx TOP-2000-2012.xlsx TOP-2000-2013.xlsx TOP-2000-2014.xlsx TOP-2000-2015.xlsx TOP-2000-2016.xlsx TOP-2000-2017.xlsx TOP-2000-2018.xlsx TOP-2000-2019.xlsx


In [3]:
# Set dict for artist replacement
replace_artist_names = dict()
replace_title_names = dict()

In [11]:
top_2000 = pd.read_excel(os.path.join(path,"excel",files[0]))
# Create merge column for artist
top_2000['artiest_merge'] = top_2000['artiest'].str.replace(pat='The ', repl = '')


# Iterate over artists to see if they are in the dict and if not add them
for artist in top_2000[top_2000['artiest'].str.match(r'^(The .*)$')
                      ][["artiest_merge","artiest"]].itertuples(index=False):
    if artist[0] not in replace_artist_names.keys():
        replace_artist_names[artist[0]] = artist[1]
        
        
# Set all strings to lower for merge
top_2000['titel_merge'] = top_2000['titel'].str.lower()

# Titles to lower
# Iterate over artists to see if they are in the dict and if not add them
for title in top_2000[["titel_merge","titel"]].itertuples(index=False):
    if title[0] not in replace_title_names.keys():
        replace_title_names[title[0]] = title[1]

# Move ranking to last column
top_2000 = top_2000[['titel_merge','artiest_merge','jaar',1999]]


# print(replace_artist_names)
# print(replace_title_names)
top_2000.head()

,titel_merge,artiest_merge,jaar,1999
0,bohemian rhapsody,Queen,1975,1
1,hotel california,Eagles,1977,2
2,child in time,Deep Purple,1972,3
3,stairway to heaven,Led Zeppelin,1971,4
4,paradise by the dashboard light,Meat Loaf,1978,5


In [12]:
replace_artist_names = dict()

for file in files[1:]:
    print(file)
    year = pd.read_excel(os.path.join(path,"Excel",file))
    
#     2019 has different column names
    if file == 'TOP-2000-2019.xlsx':
        year.columns = [2019,'titel','artiest','jaar']
    
    # Create merge column for artist
    year['artiest_merge'] = year['artiest'].str.replace(pat='The ', repl = '')
    
    # For all artist with "The" as acticle, create dict with match name and orginal name
    # Create boolean to select artists
    artist_name_the = year['artiest'].str.match(r'^(The .*)$')
    
    # Iterate over artists to see if they are in the dict and if not add them
    for artist in year[artist_name_the][["artiest_merge","artiest"]].itertuples(index=False):
        if artist[0] not in replace_artist_names.keys():
            replace_artist_names[artist[0]] = artist[1]
            
    # Drop artist name before merging
    year.drop(columns=['artiest'], inplace = True)
    
    
    
    #Set all strings to lower for merge
    year['titel_merge'] = year['titel'].str.lower()
    
    # Titles to lower
    # Iterate over artists to see if they are in the dict and if not add them
    for title in year[["titel_merge","titel"]].itertuples(index=False):
        if title[0] not in replace_title_names.keys():
            replace_title_names[title[0]] = title[1]
            
    # Drop artist name before merging
    year.drop(columns=['titel'], inplace = True)
        
    
    top_2000 = pd.merge(top_2000,year, on=['titel_merge','artiest_merge','jaar'], how='outer')

top_2000['titel_merge'] = top_2000['titel_merge'].replace(to_replace=replace_title_names)
top_2000['artiest_merge'] = top_2000['artiest_merge'].replace(to_replace=replace_artist_names)
top_2000 = top_2000.rename(columns={'artiest_merge': 'artiest',"titel_merge":"titel" })

TOP-2000-2000.xlsx
TOP-2000-2001.xlsx
TOP-2000-2002.xlsx
TOP-2000-2003.xlsx
TOP-2000-2004.xlsx
TOP-2000-2005.xlsx
TOP-2000-2006.xlsx
TOP-2000-2007.xlsx
TOP-2000-2008.xlsx
TOP-2000-2009.xlsx
TOP-2000-2010.xlsx
TOP-2000-2011.xlsx
TOP-2000-2012.xlsx
TOP-2000-2013.xlsx
TOP-2000-2014.xlsx
TOP-2000-2015.xlsx
TOP-2000-2016.xlsx
TOP-2000-2017.xlsx
TOP-2000-2018.xlsx
TOP-2000-2019.xlsx


In [13]:
# # NaNs are floats in pandas, therefore all years are of type float
# # This can be handled by an experimental functionality in pandas 1.0.1
top_2000.iloc[:,2:] = top_2000.iloc[:,2:].astype("Int64", copy=False)

In [14]:
# Check if list is now int
display(top_2000.head(100))
print(top_2000.dtypes)

,titel,artiest,jaar,1999,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Bohemian Rhapsody,Queen,1975,1,1,1,1,1,1,2,...,2,1,1,1,2,2,1,1,1,1
1,Hotel California,Eagles,1977,2,4,3,3,2,2,3,...,1,2,2,2,1,3,2,2,2,2
2,Child In Time,Deep Purple,1972,3,2,2,2,3,3,4,...,4,3,4,4,4,8,5,5,6,<NA>
3,Stairway To Heaven,Led Zeppelin,1971,4,3,4,4,4,4,5,...,5,5,3,3,3,5,3,3,4,5
4,Paradise By The Dashboard Light,Meat Loaf,1978,5,5,5,5,5,6,6,...,9,11,19,22,17,30,27,27,29,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Eternal Flame,The Bangles,1989,96,138,104,199,244,205,334,...,427,546,835,814,666,774,662,775,1120,1300
96,Strawberry Fields Forever,The Beatles,1967,97,131,180,214,207,259,261,...,247,271,308,301,395,322,387,505,587,548
97,Against All Odds (Take A Look At Me Now),Phil Collins,1984,98,152,132,153,159,170,238,...,308,394,240,307,370,393,352,282,320,269
98,Every Breath You Take,The Police,1983,99,166,167,184,259,284,434,...,351,345,356,210,330,386,318,290,286,311


titel      object
artiest    object
jaar        Int64
1999        Int64
2000        Int64
2001        Int64
2002        Int64
2003        Int64
2004        Int64
2005        Int64
2006        Int64
2007        Int64
2008        Int64
2009        Int64
2010        Int64
2011        Int64
2012        Int64
2013        Int64
2014        Int64
2015        Int64
2016        Int64
2017        Int64
2018        Int64
2019        Int64
dtype: object


In [17]:
# Save files
top_2000.to_json('top_2000.json', orient='records')
top_2000.to_csv('top_2000.csv', index = False)
top_2000.to_pickle('top_2000.pickle')

# Testing for errors

In [16]:
top_2000[(top_2000['artiest'].str.match("Andr. Hazes", case = False))&(top_2000[2019].isna())]

,titel,artiest,jaar,1999,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
2665,Diep In Mijn Hart,Andre Hazes,1982,<NA>,<NA>,1241,<NA>,1900,1125,1512,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2736,Ik Meen 't,Andre Hazes,1985,<NA>,<NA>,<NA>,1290,1800,1250,1440,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2789,Zij Gelooft In Mij,André Hazes,2000,<NA>,<NA>,<NA>,<NA>,1039,30,62,...,192,208,169,187,275,283,256,254,254,<NA>
2830,Eenzame Kerst,Andre Hazes,1976,<NA>,<NA>,<NA>,<NA>,1706,788,1408,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2913,Droomland,André Hazes & Paul de Leeuw,1993,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,663,...,630,783,599,603,702,815,1015,987,916,<NA>


In [ ]:
display(top_2000[(top_2000[2018].notna())&(top_2000[2019].isna())].sort_values(by=["titel"]))